1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [1]:
import numpy as np
import csv
int_array = np.random.randint(1,20,10)
with open("data/data_int.txt", "w") as file:
    for x in int_array:
        file.write(str(x)+", ")
    file.close()
!cat data/data_int.txt
print("\n")
matrix = np.random.rand(5,5)*10
with open("data/data_float.txt", "w") as file:
    for x in matrix:
        file.write(str(x)+"\n")
    file.close()
!cat data/data_float.txt

with open("data/data_float.txt", 'r') as file:
    txt_data = [line.strip().split() for line in file]
    file.close()

with open("data/data_csv.csv", 'w') as file:
    scrivi = csv.writer(file)
    scrivi.writerows(txt_data)
    file.close()

5, 2, 17, 13, 2, 11, 14, 5, 9, 11, 

[5.53240697 6.94697965 0.19751041 1.99494406 3.84792653]
[1.06459942 3.62869247 7.09985193 0.77774005 6.1264585 ]
[1.77659942 7.92497506 5.82998838 9.78410345 6.51114978]
[1.7777384  4.96399352 6.78837642 6.94640324 0.96190068]
[2.43361276 5.30316145 6.67774169 4.78489124 1.31800524]


2\. **JSON files**

Load the file `user_data.json`, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [2]:
import json
with open("data/user_data.json") as file:
    data = json.load(file)
    filtered = []
    for entry in data:
        if entry["CreditCardType"]=="American Express":
            filtered.append(entry)
    titoli=data[0].keys()
    file.close()
with open("data/user_data.csv", 'w') as file:
    writer = csv.DictWriter(file, fieldnames=titoli)
    writer.writeheader()
    writer.writerows(filtered)

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [3]:
import pandas as pd
data = pd.read_csv("data/mushrooms_categorized.csv")
#print(data)
class0=data[data["class"]==0]
class1=data[data["class"]==1]
for i in range(class0.shape[1]):
    media=class0.iloc[i].mean()
    print("The average in the class 0 on the", i, "-th coloumn is:", media)
print()
for i in range(class1.shape[1]):
    media=class1.iloc[i].mean()
    print("The average in the class 1 on the", i, "-th coloumn is:", media)
transfer = []
with open("data/mushrooms_categorized.csv", "r") as file:
    dati = csv.reader(file)
    for riga in dati:
        transfer.append(riga)
    file.close()
with open("data/mushrooms_categorized.json", "w") as file:
    json.dump(transfer, file)
    file.close()

The average in the class 0 on the 0 -th coloumn is: 2.391304347826087
The average in the class 0 on the 1 -th coloumn is: 2.391304347826087
The average in the class 0 on the 2 -th coloumn is: 2.1739130434782608
The average in the class 0 on the 3 -th coloumn is: 2.4347826086956523
The average in the class 0 on the 4 -th coloumn is: 2.0869565217391304
The average in the class 0 on the 5 -th coloumn is: 2.4782608695652173
The average in the class 0 on the 6 -th coloumn is: 2.1739130434782608
The average in the class 0 on the 7 -th coloumn is: 2.4782608695652173
The average in the class 0 on the 8 -th coloumn is: 2.5652173913043477
The average in the class 0 on the 9 -th coloumn is: 2.4782608695652173
The average in the class 0 on the 10 -th coloumn is: 2.0434782608695654
The average in the class 0 on the 11 -th coloumn is: 2.5652173913043477
The average in the class 0 on the 12 -th coloumn is: 2.1739130434782608
The average in the class 0 on the 13 -th coloumn is: 2.3043478260869565
The 

4\. **Reading a database**

Get the database `sakila.db` from the lecture `06_dataio.ipynb`, and import the table `actors` as a Pandas dataframe. Using the dataframe, count how many actors have a first name that begins with `A`.

*Hint:* use the Series `.str` method to apply the Python string methods to the elements of a Series, see [documentation](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html).

In [36]:
import sqlite3 as sql
conn = sql.connect('data/sakila.db')
curs = conn.cursor()
query = "SELECT * FROM actor"
results = curs.execute(query).fetchall()
actors = pd.Series(results)
print(actors)
c=0
letter="A"
for i in range(len(actors)):
    entry = str(actors[i]).split(",")
    name = entry[1]
    if name[2]==letter:
        c+=1
print(c, "actors have a name which start with", letter)

0         (1, PENELOPE, GUINESS, 2019-02-16 18:17:33)
1            (2, NICK, WAHLBERG, 2019-02-16 18:17:33)
2                 (3, ED, CHASE, 2019-02-16 18:17:33)
3           (4, JENNIFER, DAVIS, 2019-02-16 18:17:33)
4      (5, JOHNNY, LOLLOBRIGIDA, 2019-02-16 18:17:33)
                            ...                      
195          (196, BELA, WALKEN, 2019-02-16 18:17:33)
196           (197, REESE, WEST, 2019-02-16 18:17:33)
197          (198, MARY, KEITEL, 2019-02-16 18:17:33)
198        (199, JULIA, FAWCETT, 2019-02-16 18:17:33)
199         (200, THORA, TEMPLE, 2019-02-16 18:17:33)
Length: 200, dtype: object
13 actors have a name which start with A


5\. **Reading the credit card numbers**

Get the binary file named `credit_card.dat` from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [88]:
data = np.genfromtxt("data/credit_card.dat", dtype="str")
for i in range(len(data)):
    numero=""
    stringa = data[i]
    for j in range((len(stringa)-4)//6):
        num=int(stringa[6*j:6*j+6], 2)
        numero=numero+chr(num)
    print(numero)

7648 5673 3775 2271
3257 8247 3354 2266
2722 0001 4011 6652
0661 3063 3742 3150
0432 1608 1462 4742
5827 2027 8785 7303
5774 8528 2087 1117
8140 1210 6352 2845
5764 1133 7301 7100
6456 1737 4126 6726
1228 8631 7382 0000
7051 0160 5374 3166
0618 3587 1630 6376
1545 5454 7444 5636
6735 3116 3202 6834
7287 5011 1547 8413
7033 2607 3328 4200
2568 5244 1874 5024
1684 2253 7570 7118
0672 2576 0575 6631
6332 8353 8787 1340
1813 3361 1175 4211
2477 6450 8840 2368
5512 3505 2563 1326
3083 7882 0621 0025
4521 5148 8045 0334
7563 3654 8713 5787
8324 2664 0476 5561
0565 2504 7168 3510
5107 5507 1767 0738
2462 1821 2448 1443
2788 0638 6861 6554
5851 5873 5474 0547
0670 1004 4013 2655
5874 5506 3048 0806
2805 5401 8462 1260
5083 8406 6310 1862
1076 1445 3013 2266
8440 4804 4844 5277
4758 6141 0686 1387
7586 0675 0315 2568
2544 1258 7432 5165
3474 5023 4434 5626
1410 0270 0434 5086
7315 4446 1104 4215
0224 7742 8300 0266
0170 2700 3145 0640
2006 2437 8054 1600
8142 4055 1776 0026
3026 7380 1241 1084


6\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [37]:
from IPython.display import Image
Image("images/data_format.png")

FileNotFoundError: No such file or directory: 'images/data_format.png'

FileNotFoundError: No such file or directory: 'images/data_format.png'

<IPython.core.display.Image object>

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, "pack" the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_dataio.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?